# EEP 153 Proj 2: Group Sylvia Lane

## Minimum Cost Diet

**Topic & Goals**

- Our project focused on determining the most affordable grocery store for a diabetes-friendly diet.

- Population of interest: People diagnosed with diabetes from ages 19-30 (M&F), with an ideal community of Berkeley students.

### Import Libraries & Read Datasets

In [1]:
# Install packages
!pip install -r requirements.txt >/dev/null
!pip install eep153_tools >/dev/null
!pip install gspread_pandas >/dev/null

# Import libraries
import numpy as np
import pandas as pd
import warnings
import fooddatacentral as fdc
from  scipy.optimize import linprog as lp
from eep153_tools.sheets import read_sheets

# API Key
apikey = "ztQPaunUYOUORb4qgWsBdjS2VDpKaYrwYmMP9LXV"

In [2]:
# Read diet minimum and maximum data
diet_min = pd.read_excel("Dietary Requirements.xlsx",sheet_name='diet_minimums',index_col=0)
diet_min

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [28]:
diet_max = pd.read_excel("Dietary Requirements.xlsx",sheet_name='diet_maximums',index_col=0)
diet_max

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
"Sodium, Na",UL,1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Energy,NaN,2500,2500,2500,2800,3000,3100,3100,3100,3100,3100,3100,3100,3100


### [A] Description of population of interest

Our group are interested in minimum cost diet for **diagnosed diabetes patients in Berkeley**.

### [A] Dietary Reference Intakes
A function that takes as arguments the characteristics of a person (e.g., age, sex) and returns a pandas.Series of Dietary Reference Intakes (DRI's) or Recommended Daily Allowances(RDA) of a variety of nutrients appropriate for your population of interest

In [4]:
def dietary_ref_intake(age,sex):
    """Takes in age and sex, and returns the dietary reference intake for the chosen population"""

    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(diet_min[col])  

In [5]:
# Example of DRI's for a male aged 22
dietary_ref_intake(age=22,sex='M')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Name: M 19-30, dtype: float64

### [A] Data on prices for different foods
We construct a google spreadsheet of the prices of around 40 different food products from 5 different grocery stores around Berkeley (Safeway, Trader Joe's, Amazon Fresh, Berkeley Bowl, Sprout's). The food products we chose fall into 3 distinct categories:
- **Non-starchy vegetables**: Broccoli, Cauliflower, Green beans, Cabbage, Carrots
- **Carb foods**: Pasta (whole grain), Rice (whole grain), Sweet potato, Beans (kidney beans, black beans, chickpeas), Fruits (blueberries, strawberries, cherries, pears, apricots, kiwi, plums, peaches, apples, oranges), Yogurt (Greek), Milk (Unsweetened almond or soy)
- **Protein foods**: Chicken (breast), Eggs, Turkey (white meat, if possible), Beef (sirloin), Tofu

In [11]:
# Link to our google spreadsheet
food = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRDIOWlxta4m4S6RldUFn2srNWYSCo9WeNh3CBKIrdpW9c6uJuxqVcBJzsxlRxTNOaAndUq9M1Ksscw/pub?output=xlsx'

# helper function to read sheets of different stores
def read_sheet(store):
    df = pd.read_excel(food,sheet_name=store)
    df = df.iloc[:,:5].dropna(subset=['FDC'])
    df = df.reset_index(drop=True)
    df['FDC'] = df['FDC'].astype(int)
    return df

In [12]:
sprouts = read_sheet('Sprouts')
safeway = read_sheet('Safeway')
trader_joes = read_sheet('Trader Joes')
whole_foods = read_sheet('Whole Foods')
berkeley_bowl = read_sheet('Berkeley Bowl')

In [13]:
# Example sheet from Sprouts 
sprouts

,Food,FDC,Quantity,Units,Price
0,"Spinach, raw",1999633,9.0,oz,2.50
1,"Mushrooms, shiitake, raw",1999628,1.0,oz,1.50
2,"Carrots, raw",2258586,1.0,lb,1.49
3,"Squash, summer, zucchini, includes skin, raw",169291,1.0,lb,1.34
4,"Broccoli, raw",170379,1.0,lb,3.49
5,"Tomatoes, red, ripe, raw, year round average",170457,1.0,lb,1.99
6,"Cabbage, red, raw",169977,1.0,lb,0.99
7,"Avocados (large Hass), raw, California",171706,7.6,oz,2.50
8,"Blueberries, raw",171711,6.0,oz,3.99
9,"Grapefruit, raw, white, California",174677,8.0,oz,1.50


**Units & Prices**

Now, the prices we observe can be for lots of different quantities and units. The FDC database basically wants everything in either hundreds of grams (hectograms) or hundreds of milliliters (deciliters).
We use the units function to convert all foods to either deciliters or hectograms, to match FDC database:

In [14]:
# Unit Conversion
def convert(df):
    # Convert food quantities to FDC units
    df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    df['FDC Price'] = df['Price']/df['FDC Quantity']

    df.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    Prices = df.groupby('Food',sort=False)['FDC Price'].min()

    return Prices

In [15]:
sprouts_price = convert(sprouts)
safeway_price = convert(safeway)
#trader_joes_price = convert(trader_joes)
whole_foods_price = convert(whole_foods)
berkeley_bowl_price = convert(berkeley_bowl)

/Users/eddieguo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/Users/eddieguo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/Users/eddieguo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/Users/eddieguo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1983: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


### [A] Nutritional content of different foods
Now we have a list of foods with prices. Do lookups on USDA database to get nutritional information.

In [17]:
# helper function to get nutritional info
def nutrient(store):
    df = read_sheet(store)
    D = {}
    count = 0
    for food in  df.Food.tolist():
        try:
            FDC = df.loc[df.Food==food,:].FDC[count]
            count+=1
            D[food] = fdc.nutrients(apikey,FDC).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
    return pd.DataFrame(D,dtype=float)

In [21]:
sprouts_nutrient = nutrient('Sprouts')
safeway_nutrient = nutrient('Safeway')
trader_joes_nutrient = nutrient('Trader Joes')
whole_foods_nutrient = nutrient('Whole Foods')
berkeley_bowl_nutrient = nutrient('Berkeley Bowl')

### [A] Solution

Here we use the function `solve_subsistence_problem` defined in lecture. By isolating the logic of constructing and solving the subsistence problem into a stand-alone function we reduce the scope for bugs, and this modular approach at the same time makes testing easier.

Recall that the mathematical problem we are trying to solve is
$$
    \min_x p'x
$$
such that
$$
     Ax \geq b
$$

In [22]:
def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [25]:
def match(store):
    if store == 'Sprouts':
        return sprouts_price, sprouts_nutrient
    if store == 'Safeway':
        return safeway_price, safeway_nutrient
    if store == 'Trader Joes':
        return trader_joes_price, trader_joes_nutrient
    if store == 'Whole Foods':
        return whole_foods_price, whole_foods_nutrient
    if store == 'Berkeley Bowl':
        return berkeley_bowl_price, berkeley_bowl_nutrient

In [26]:
def solution(store):
    Prices, FoodNutrients = match(store)
    group = 'M 19-30'
    tol = 1e-6

    result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

    print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

    # Put back into nice series
    diet = result.diet

    print("\nDiet (in 100s of grams or milliliters):")
    print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
    print()

    tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
    print("\nWith the following nutritional outcomes of interest:")
    print(tab)
    print()

    print("\nConstraining nutrients are:")
    excess = tab.diff(axis=1).iloc[:,1]
    print(excess.loc[np.abs(excess) < tol*100].index.tolist())

In [29]:
solution('Sprouts')

Cost of diet for M 19-30 is $9.01 per day.


Diet (in 100s of grams or milliliters):
Spinach, raw                                                                                         1.669801
Carrots, raw                                                                                        12.745600
Broccoli, raw                                                                                        1.140527
Cereals, oats, regular and quick, unenriched, cooked with water, without salt                        1.825771
Beverage, almond milk, unsweetended, shelf stable                                                    2.184875
milk , reduced fat, fluid, 2% milkfat, with added nonfat milk solids and vitamin A and vitamin D     6.315761
Egg, whole, raw, fresh                                                                               0.000012
Beans, black, mature seeds, raw                                                                      0.638351
dtype: float64


With the following